In [ ]:
from IPython.display import display, HTML
import numpy as np
import pandas as pd
import sqlalchemy as sa
import matplotlib.pyplot as plt
import datetime
import random
import os

In [ ]:
server = 'cs-eur-research-dp-olap-synw.sql.azuresynapse.net'
database = 'sqlpool01'
username = 'sqladminuser'
password = 'BeanieBoblets1234'   
driver= '{ODBC Driver 18 for SQL Server}'
port = os.getenv('PORT',default=1433)

In [ ]:
connection_url = sa.engine.URL.create(
    "mssql+pyodbc",
    username=username,
    password=password,
    host=server,
    database=database,
    query={
        "driver": "ODBC Driver 18 for SQL Server",
        "autocommit": "True",
    },
)
engine = sa.create_engine(connection_url, fast_executemany=True)

In [ ]:
df_soh = pd.read_sql_table('sales_order_header', engine, schema="AdventureWorks")
print(df_soh.columns)
display(df_soh)

In [ ]:
df_sod = pd.read_sql_table('sales_order_detail', engine, schema="AdventureWorks")
print(df_sod.columns)
display(df_sod)

In [ ]:
#cnxn = pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
#cursor = cnxn.cursor()

In [ ]:
#sql = "SELECT * from AdventureWorks.sales_order_header"
#df_soh = pd.read_sql(sql, cnxn)
#print(df_soh.columns)
#display(df_soh)

In [ ]:
#sql = "SELECT * from AdventureWorks.sales_order_detail"
#df_sod = pd.read_sql(sql, cnxn)
#print(df_sod.columns)
#display(df_sod)

In [ ]:
# Get distinct customers
df_customers = df_soh[['AccountNumber', 'CustomerID', 'ShipToAddressID', 'BillToAddressID', 'ShipMethod', 'CreditCardApprovalCode']].copy()
df_customers.drop_duplicates(inplace=True)
display(df_customers)


In [ ]:
# Get distinct products
df_products = df_sod[['ProductID', 'UnitPrice',	'UnitPriceDiscount']].copy()
df_products.drop_duplicates(inplace=True)
display(df_products)

In [ ]:
# Create stochastic increasing sales model 
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2022, 4, 2)
date_delta = end_date - start_date
date_delta.days

mu = 0.002
sigma = 0.01
start_qty_orders_per_day = 5

np.random.seed(0)
returns = np.random.normal(loc=mu, scale=sigma, size=date_delta.days)
qty_orders_per_day = start_qty_orders_per_day*(1+returns).cumprod()

print(qty_orders_per_day)
plt.plot(qty_orders_per_day)

In [ ]:
qty_orders_per_day= [int(qty) for qty in qty_orders_per_day]
print(qty_orders_per_day)

In [ ]:
sales_order_id = 0
sales_order_detail_id = 0
revision_number = 2


In [ ]:
# Create new enlarged dataframes as copies oof originals but only column/datatype definitions
df_soh_enlarged = df_soh.iloc[:0].copy()
df_sod_enlarged = df_sod.iloc[:0].copy()
print(df_soh_enlarged.columns)
print(df_sod_enlarged.columns)


In [ ]:
for _, i in enumerate(range(date_delta.days)):
    order_date = start_date + datetime.timedelta(days=i)
    ship_date = start_date + datetime.timedelta(days= i + 1)
    due_date = start_date + datetime.timedelta(days= i + 7)
    order_date_as_timestamp = pd.Timestamp(order_date)
    ship_date_as_timestamp = pd.Timestamp(ship_date)
    due_date_as_timestamp = pd.Timestamp(due_date)
    
    for _, ord in enumerate(range(qty_orders_per_day[i])):
        sales_order_id += 1
        sales_order_number = 'SO' + str(sales_order_id)
        sales_order_sub_total = 0

        number_order_detail_lines = random.randint(1, 3)
        df_random_customer = df_customers.sample(n=1,replace=True)  # Select 1 random customer and allow this customer to be reselected

        for _, det in enumerate(range(number_order_detail_lines)):
            sales_order_detail_id += 1
            df_random_product = df_products.sample(n=1,replace=True)  # Select 1 random product and allow it to be reselected
            product_id = df_random_product['ProductID'].iloc[0]
            unit_price = df_random_product['UnitPrice'].iloc[0]
            unit_price_discount = df_random_product['UnitPriceDiscount'].iloc[0]
            order_qty = random.randint(1, 5)
            line_total = (unit_price - (unit_price * unit_price_discount)) * order_qty
            
            sales_order_sub_total += line_total
            
            new_detail = {'SalesOrderID': sales_order_id, 
                          'SalesOrderDetailID': sales_order_detail_id, 
                          'OrderQty': order_qty, 
                          'ProductID':product_id,
                          'UnitPrice': unit_price,
                          'UnitPriceDiscount': unit_price_discount, 
                          'LineTotal': line_total, 
                          'rowguid': '',
                          'ModifiedDate':order_date_as_timestamp}
            
            df_sod_enlarged = df_sod_enlarged.append(new_detail, ignore_index=True)
        
        tax_amount = sales_order_sub_total * 0.079
        freight_amount = sales_order_sub_total * 0.02
        
        new_header = {'SalesOrderID': sales_order_id, 
                      'RevisionNumber': 2, 
                      'OrderDate': order_date_as_timestamp, 
                      'DueDate': due_date_as_timestamp, 
                      'ShipDate': ship_date_as_timestamp,
                      'Status': 5, 
                      'OnlineOrderFlag': 1, 
                      'SalesOrderNumber': sales_order_number, 
                      'PurchaseOrderNumber': "",
                      'AccountNumber': df_random_customer['AccountNumber'].iloc[0], 
                      'CustomerID': df_random_customer['CustomerID'].iloc[0], 
                      'ShipToAddressID': df_random_customer['ShipToAddressID'].iloc[0],
                      'BillToAddressID': df_random_customer['BillToAddressID'].iloc[0],
                      'ShipMethod': "CARGO TRANSPORT", 
                      'CreditCardApprovalCode': '5', 
                      'SubTotal': sales_order_sub_total, 
                      'TaxAmt': tax_amount,
                      'Freight': freight_amount,
                      'TotalDue': sales_order_sub_total + tax_amount + freight_amount, 
                      'Comment': "", 
                      'rowguid': "", 
                      'ModifiedDate': order_date_as_timestamp, 
                      'OrderYear': "",
                      'OrderMonth': ""}
        
        df_soh_enlarged = df_soh_enlarged.append(new_header, ignore_index=True)

In [ ]:
display(df_soh_enlarged[:5])

In [ ]:
print(len(df_soh_enlarged))

In [ ]:
display(df_sod_enlarged[:5])

In [ ]:
print(len(df_sod_enlarged))

In [ ]:
df_soh_enlarged.to_sql("sales_order_header_enlarged", engine, schema="AdventureWorks", if_exists='replace', index=False, dtype={
    'SalesOrderNumber': sa.types.NVARCHAR(255),
    'PurchaseOrderNumber': sa.types.NVARCHAR(255),
    'AccountNumber': sa.types.NVARCHAR(255),
    'ShipMethod': sa.types.NVARCHAR(255),
    'CreditCardApprovalCode': sa.types.NVARCHAR(255),
	'Comment': sa.types.NVARCHAR(255),
	'rowguid': sa.types.NVARCHAR(255),
	'ModifiedDate': sa.types.NVARCHAR(255), 
	'OrderYear': sa.types.NVARCHAR(255), 
	'OrderMonth': sa.types.NVARCHAR(255)
})

In [ ]:
df_sod_enlarged.to_sql("sales_order_detail_enlarged", engine, schema="AdventureWorks", if_exists='replace', index=False, dtype={
	'rowguid': sa.types.NVARCHAR(255),
})

In [ ]:
for index, row in df_soh_enlarged.iterrows():
     cursor.execute("INSERT INTO [AdventureWorks].[sales_order_header_enlarged] ('SalesOrderID', 'RevisionNumber', 'OrderDate', 'DueDate', 'ShipDate', 'Status', 'OnlineOrderFlag', 'SalesOrderNumber', 'PurchaseOrderNumber','AccountNumber', 'CustomerID', 'ShipToAddressID', 'BillToAddressID', 'ShipMethod', 'CreditCardApprovalCode', 'SubTotal', 'TaxAmt', 'Freight', 'TotalDue', 'Comment', 'rowguid', 'ModifiedDate', 'OrderYear', 'OrderMonth') values(?,?,?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
          row.SalesOrderID, row.RevisionNumber, row.OrderDate, row.DueDate, row.ShipDate, row.Status, row.OnlineOrderFlag, row.SalesOrderNumber, row.PurchaseOrderNumber,row.AccountNumber, row.CustomerID, row.ShipToAddressID, row.BillToAddressID, 
          row.ShipMethod, row.CreditCardApprovalCode, row.SubTotal, row.TaxAmt, row.Freight, row.TotalDue, row.Comment, row.rowguid, row.ModifiedDate, row.OrderYear, row.OrderMonth)